In [1]:
import numpy as np
import cv2 
from skimage import io
import os

In [2]:
def preprocessing(img) :
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # change bgr image to gray scale image
    return (img, img_gray)

In [3]:
sift = cv2.SIFT_create() # find important point from book
bf = cv2.BFMatcher() 

In [4]:
def feature_object_detection(template_img, template_gray, query_img, query_gray, min_match_number) :
    template_kpts, template_desc = sift.detectAndCompute(template_gray, None)
    query_kpts, query_desc = sift.detectAndCompute(query_gray, None)
    matches = bf.knnMatch(template_desc, query_desc, k=2)
    good_matches = list()
    good_matches_list = list()
    for m, n in matches :
        if m.distance < 0.8*n.distance :
            good_matches.append(m)
            good_matches_list.append([m])
    
    if len(good_matches) > min_match_number :
        src_pts = np.float32([ template_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
        dst_pts = np.float32([ query_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)

        H, inlier_masks = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 0) # H RANSAC
        
        h, w = template_img.shape[:2]
        template_box = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2)
        transformed_box = cv2.perspectiveTransform(template_box, H)

        detected_img = cv2.polylines(query_img, [np.int32(transformed_box)], True, (0,0,255), 2, cv2.LINE_AA)
        drawmatch_img = cv2.drawMatchesKnn(template_img, template_kpts, detected_img, query_kpts, good_matches_list, None, flags=2, matchesMask=inlier_masks)
        return detected_img, drawmatch_img
    else :

        return
    

In [5]:
template_img_url = "./images/final_exam/Templates/Template-3.png"
template_img = io.imread(template_img_url)
template_img, template_gray = preprocessing(template_img)

In [6]:
vid = cv2.VideoCapture('./videos/final_exam/Dataset-1/left_output-1.avi')


while vid.isOpened() :
    ret, frame = vid.read()

    if ret :
        vid_gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        detected, drawmatch =  feature_object_detection(template_img, template_gray, frame, vid_gray, 10)
        cv2.imshow('Video frame', frame)
        if cv2.waitKey(int(33)) & 0xFF == ord('q') : 
            break
    else :
        break
vid.release()
cv2.destroyAllWindows()